In [9]:
import sys
from importlib import reload

try:
    reload(sys.modules['utils']);
    reload(sys.modules['mds.cmds']);
    reload(sys.modules['mds.lmds']);
    reload(sys.modules['trainHelper']);
    reload(sys.modules['lossFunction']);
except:
    pass

import trainHelper
import utils
from mds.lmds import landmarkMDS
from mds.cmds import classicalMDS
from mds.fastmap import fastmap
import numpy
from torch.utils.data import DataLoader
import torch
from torch import nn, Tensor
import lossFunction as lossF
from model.AutoEncoder import AutoEncoder
from model.Linear import Linear
import model.RNN as rnn
import os.path
import glob
import dataSource

torch.set_default_tensor_type('torch.DoubleTensor')

In [10]:
sample_space = (1000, 1)
ss, N, d = 3200, 10, 2

test_batch = 1000
test_data = dataSource.generate_rand_DM(
                N=N,
                sample_size=test_batch, 
                sample_space=sample_space, isInt=True)

test_data = dataSource.generate_euclidean_DM(
                N=N, d=2,
                sample_size=test_batch, 
                sample_space=sample_space, isInt=True)

test_data = test_data.view(test_batch, 1, N, N)
test_data = utils.minmax_norm(test_data, dmin=0)[0]

In [11]:

# %%
cmds_loss, fastmap_loss, lmds_loss = [], [], []

test_data = utils.load_variable('data\euclidean.dat')
test_data = utils.minmax_norm(test_data, dmin=0)[0]

for d in test_data:

    d1 = numpy.array(d[0].data) ** 2

    cmds_rs = classicalMDS(d1, 2)
    cmds_rs = torch.tensor(cmds_rs)
    cmds_dm = utils.get_distance_matrix(cmds_rs) ** 0.5 
    cmds_dm = utils.minmax_norm(cmds_dm, dmin=0)[0]
    
    cmds_loss.append(torch.mean((cmds_dm - d)** 2))

    lmds_rs = landmarkMDS(d1, L=5, D=2)
    lmds_rs = torch.tensor(lmds_rs)
    lmds_dm = utils.get_distance_matrix(lmds_rs) ** 0.5
    lmds_dm = utils.minmax_norm(lmds_dm, dmin=0)[0]

    lmds_loss.append(torch.mean((lmds_dm - d)** 2))

    fastmap_rs = fastmap(d1, 2)
    fastmap_rs = torch.tensor(fastmap_rs)
    fastmap_dm = utils.get_distance_matrix(fastmap_rs) ** 0.5
    fastmap_dm = utils.minmax_norm(fastmap_dm, dmin=0)[0]

    fastmap_loss.append(torch.mean((fastmap_dm - d)** 2))


print("cmds_loss: \t", 
        torch.tensor(cmds_loss).mean(), "|" , 
        len(cmds_loss), "success")

print("lmds_loss: \t", 
        torch.tensor(lmds_loss).mean(), "|" , 
        len(lmds_loss), "success")

print("fastmap_loss: \t", 
        torch.tensor(fastmap_loss).mean(), "|" , 
        len(fastmap_loss), "success")

cmds_loss: 	 tensor(1.9503e-31) | 1000 success
lmds_loss: 	 tensor(0.0292) | 1000 success
fastmap_loss: 	 tensor(0.0448) | 1000 success


In [12]:
max(cmds_loss)

tensor(6.3726e-30)

In [27]:

def linear_test_model(model, preprocess, test_data, target_dm):

    t_data = preprocess(test_data)
    t_data = utils.minmax_norm(t_data, dmin=0)[0]

    model_rs = model(t_data)
    model_rs = model_rs.view(model_rs.size()[0], *target_dm)

    model_dm = utils.get_distance_matrix(model_rs) ** 0.5
    model_dm = utils.minmax_norm(model_dm, dmin=0)[0]

    loss = (model_dm - test_data.view_as(model_dm)) ** 2

    return torch.mean(loss)
    

linear_result_score = []

all_lrs = []

to_loop = set(glob.glob('backup/Coord_Linear*.model'))

for filepath in to_loop:
    h: trainHelper.TrainHelper = utils.load_variable(filepath)
    linear_result_score.append([
        filepath, 
        linear_test_model(h.model, h.preprocess, test_data, (N, 2))
        ])

linear_result_score = sorted(linear_result_score, key=lambda x: x[1])

for rss in linear_result_score[:]:
    print(rss[0], "   \t\t\t|", rss[1].data)

# all_lrs.append(['matrix_LReLU_900.model', '5', '64', '0.0691536749058093'])


backup\Coord_Linear_1_64_D_MSE_500.model    			| tensor(0.0639)
backup\Coord_Linear_1_64_D_MSE_700.model    			| tensor(0.0639)
backup\Coord_Linear_1_64_D_MSE_600.model    			| tensor(0.0639)
backup\Coord_Linear_1_64_D_MSE_400.model    			| tensor(0.0642)
backup\Coord_Linear_1_64_D_MSE_200.model    			| tensor(0.0646)
backup\Coord_Linear_1_64_D_MSE_300.model    			| tensor(0.0646)
backup\Coord_Linear_2_80_D_MSE_200.model    			| tensor(0.0648)
backup\Coord_Linear_2_64_D_MSE_400.model    			| tensor(0.0648)
backup\Coord_Linear_2_64_D_MSE_300.model    			| tensor(0.0649)
backup\Coord_Linear_2_64_D_MSE_500.model    			| tensor(0.0650)
backup\Coord_Linear_2_64_D_MSE_600.model    			| tensor(0.0650)
backup\Coord_Linear_2_64_D_MSE_700.model    			| tensor(0.0650)
backup\Coord_Linear_1_80_D_MSE_300.model    			| tensor(0.0651)
backup\Coord_Linear_1_80_D_MSE_600.model    			| tensor(0.0651)
backup\Coord_Linear_1_80_D_MSE_500.model    			| tensor(0.0651)
backup\Coord_Linear_2_80_D_MSE_300.model

In [ ]:

rs = numpy.array(all_lrs)

for i in [32, 64, 96]:
    t = rs[rs[:, 2] == str(i)]
    t = t[t[:, 0] == 'distance_LReLU_900.model']

    t = t[t[:, 1].argsort()]
    
    print(t)

    plt.ylabel("loss")
    plt.xlabel("number of layers")

    plt.plot(t[:, 1], t[:, 3].astype('double'), label=str(i))


plt.legend()


In [ ]:
to_loop = set(glob.glob('backup/Coord*_96_distance_*_900.model')) - set(glob.glob('backup/*eign*.model'))

for filepath in sorted(to_loop):
    h: trainHelper.TrainHelper = utils.load_variable(filepath)
    t = h.records
    print(t)
    plt.plot(range(900), t['loss_sum'],label=filepath[20])

plt.xlabel('SSE')
plt.ylabel('Epoch')

plt.legend()
# plt.xscale('log')

In [ ]:

def recon_test_model(model, preprocess, test_data, target_dm):

    t_data = preprocess(test_data)
    t_data = utils.minmax_norm(t_data)[0]

    model_rs = model(t_data ** 2)
    model_rs = utils.minmax_norm(model_rs)[0]

    model_dm =  utils.unvectorize_distance(model_rs)

    score = []

    batch = model_dm.size()[0]

    for i in range(batch):
        dm = model_dm[i]
        # t = test_data[i].view_as(dm)

        # score.append(torch.mean((dm - t) ** 2))
        # continue

        rs = classicalMDS(dm.data, 2)
        rs = torch.tensor(rs)

        rs_dm = utils.get_distance_matrix(rs) ** 0.5
        rs_dm = utils.minmax_norm(rs_dm, dmin=0)[0]

        loss = (rs_dm - test_data[i].view_as(rs_dm)) ** 2

        score.append(torch.mean(loss))

    return torch.tensor(score).mean()


recon_result_score = []

for filepath in glob.iglob('backup/Recon_*900.model'):
    h: trainHelper.TrainHelper = utils.load_variable(filepath)
    recon_result_score.append([
        filepath, 
        recon_test_model(h.model, h.preprocess, test_data, (N, 2))
        ])
    
recon_result_score = sorted(recon_result_score, key=lambda x: x[1])

for rss in recon_result_score[:]:
    print(rss[0], "   \t\t\t|", rss[1].data)

In [ ]:
all_lrs_recon = []

for rs in recon_result_score:
    t = rs[0]
    if t[13] == 'L':
        all_lrs_recon.append([t[13], t[20], t[22:24], float(rs[1])])
    else:
        all_lrs_recon.append([t[13], t[16], t[18:20], float(rs[1])])


In [ ]:

rs = numpy.array(all_lrs_recon)

for m in ['A']:
    for i in [32, 64, 96]:
        t = rs[rs[:, 2] == str(i)]
        t = t[t[:, 0] == m]

        t = t[t[:, 1].argsort()]
        
        print(t)

        plt.ylabel("loss")
        plt.xlabel("number of layers")

        plt.plot(t[:, 1], t[:, 3].astype('double'), label=str(i))


plt.legend()


In [ ]:
class CoordsToDMLoss(nn.Module):

    def __init__(self, N, d):
        super(CoordsToDMLoss, self).__init__()

        self.N = N
        self.d = d

    def forward(self, rs, target_dm):

        batch = target_dm.size()[0]

        rs = rs.view(batch, self.N, self.d)

        model_dm = utils.get_distance_matrix(rs)
        model_dm = utils.minmax_norm(model_dm, dmin=0)[0]

        target_dm = torch.pow(target_dm, 2)
        target_dm = utils.minmax_norm(target_dm, dmin=0)[0]

        loss = model_dm - target_dm.view_as(model_dm)
        loss = torch.pow(loss, 2)

        loss = loss.view(loss.size()[0], 1, -1)
        return torch.sum(torch.sum(loss, dim=2) ** 2)


def preprocess(x):
    return x.clone().detach().requires_grad_(True)


def DM_distance(x):
    x = utils.vectorize_distance_from_DM(x)
    return preprocess(x)


class ReconLoss(nn.Module):

    def __init__(self):

        super(ReconLoss, self).__init__()

    def forward(self, rs, target_dm):

        model_dm = rs
        model_dm = utils.minmax_norm(model_dm, dmin=0)[0]

        target_dm = torch.pow(target_dm, 2)
        target_dm = utils.minmax_norm(target_dm, dmin=0)[0]

        target_dm = utils.vectorize_distance_from_DM(target_dm)

        loss = model_dm - target_dm.view_as(model_dm)
        loss = torch.pow(loss, 2)

        loss = loss.view(loss.size()[0], 1, -1)
        return torch.sum(torch.sum(loss, dim=2) ** 2)


In [ ]:
utils.eignmatrix(d1.view(2, 3, 3))

In [ ]:
h = utils.load_variable('backup/bk_squre/Linear_SGD_2_32_distance_LReLU_300.model')

In [ ]:
h.records

In [ ]:
from gradient_check.bad_grad_viz import view_gradient

l = utils.get_distance_matrix())
view_gradient(torch.sum(l))

In [ ]:
import tensorflow as tf

In [ ]:
import torch

eps = torch.tensor(1e-8, requires_grad=True)

def sammon_loss(result, target):
    
    result = torch.triu(result)
    target = torch.triu(target)
    
    loss = torch.pow(target - result, 2)

    a = torch.div(loss, target + eps)
    a = torch.sum(a)

    b = torch.sum(target)
    
    print(a, b)
    
    return torch.div(a, b)


target = torch.tensor([
        [0, 1, 2],
        [1, 0, 3],
        [2, 3, 0.0]
    ], requires_grad=True)

rs = torch.tensor([
        [0, 4, 2],
        [4, 0, 3],
        [2, 3, 0.0]
    ], requires_grad=True)


loss = sammon_loss(rs, target)
loss

In [ ]:
a = torch.triu(target)
a = torch.nonzero(a)
print(a)


In [ ]:
mytriu = torch.triu(target)
idx = torch.nonzero(torch.triu(torch.ones(*mytriu.size())).view(-1)).view(-1)
vec = mytriu.view(-1)[idx]

In [ ]:
from gradient_check.bad_grad_viz import view_gradient

view_gradient(torch.sum(loss))

In [8]:
model = Linear([3, 16, 16, 1])
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
cal = lambda x: x[0] * x[1] + x[2]

for i in range(10000):
    optim.zero_grad()

    x = torch.rand(3)
    loss = model(torch.tensor(x.data, requires_grad=True)) - cal(x.data)
    loss = torch.pow(loss, 2)

    print(loss)
    
    loss.backward()
    optim.step()

ackward0>)
tensor([1.1502e-05], grad_fn=<PowBackward0>)
tensor([0.0008], grad_fn=<PowBackward0>)
tensor([8.1124e-05], grad_fn=<PowBackward0>)
tensor([2.9526e-05], grad_fn=<PowBackward0>)
tensor([1.1923e-05], grad_fn=<PowBackward0>)
tensor([0.0003], grad_fn=<PowBackward0>)
tensor([0.0004], grad_fn=<PowBackward0>)
tensor([3.4663e-09], grad_fn=<PowBackward0>)
tensor([3.3728e-05], grad_fn=<PowBackward0>)
tensor([0.0003], grad_fn=<PowBackward0>)
tensor([0.0004], grad_fn=<PowBackward0>)
tensor([1.7079e-06], grad_fn=<PowBackward0>)
tensor([0.0004], grad_fn=<PowBackward0>)
tensor([7.3488e-06], grad_fn=<PowBackward0>)
tensor([0.0051], grad_fn=<PowBackward0>)
tensor([0.0002], grad_fn=<PowBackward0>)
tensor([0.0018], grad_fn=<PowBackward0>)
tensor([0.0013], grad_fn=<PowBackward0>)
tensor([0.0021], grad_fn=<PowBackward0>)
tensor([0.0011], grad_fn=<PowBackward0>)
tensor([0.0011], grad_fn=<PowBackward0>)
tensor([5.7970e-05], grad_fn=<PowBackward0>)
tensor([0.0014], grad_fn=<PowBackward0>)
tensor([0.

In [13]:
model(torch.tensor([0.1, 0.2, 0.3])), cal([0.1, 0.2, 0.3])

(tensor([0.3054], grad_fn=<AddBackward0>), 0.32)

In [14]:
round(1.0000002, 3)

1.0